In [1]:
import torch
import pandas as pd
from transformers import AutoTokenizer

In [ ]:
base_directory = './'
model = torch.load(base_directory + 'transformers_french_to_english.pt')
print(model)

In [2]:
test_df = pd.read_csv('test_preprocess.csv')
test_df

,src,tar
0,That was pathetic.,C'était pathétique.
1,What is the theme of his latest novel?,Quel est le sujet de son dernier roman ?
2,Do you like salmon?,Aimes-tu le saumon ?
3,How do you talk to women?,Comment parler aux femmes ?
4,In autumn the leaves turn yellow.,"En automne, les feuilles virent au jaune."
...,...,...
23269,Wash the dishes.,Lavez la vaisselle !
23270,I know how badly you want it.,Je sais combien tu le veux.
23271,I'm going mad.,Je suis en train de devenir fou.
23272,"Tom was your guitar teacher, wasn't he?","Tom fut ton professeur de guitare, n'est-ce pas ?"


In [3]:
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
tokenizer.add_special_tokens({'bos_token':'<s>'})

1

In [4]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.src = self.tokenizer(list(self.data['src']), padding=True, truncation=True, max_length = self.max_len, return_tensors='pt').input_ids
        self.tar = self.tokenizer(['<s>' + s for s in self.data['tar']], padding=True, truncation=True, max_length = self.max_len, return_tensors='pt').input_ids
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        return self.src[idx], self.tar[idx]

In [5]:
test_ds = CustomDataset(test_df, tokenizer, 120)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=64)
test_dl

In [6]:
test_src, test_tar = next(iter(test_dl))
print(test_src.shape)
print(test_src[0])
print(tokenizer.decode(test_src[0]))
print(test_tar.shape)
print(test_tar[0])
print(tokenizer.decode(test_tar[0]))

torch.Size([64, 44])
tensor([  466,    47,     3, 27826,     5,     1,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
That was pathetic.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
torch.Size([64, 67])
tensor([32100,   205,    31,  6449,  2071, 12635,     5,     1,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [11]:
tokenizer.encode('That was pathetic')

[466, 47, 3, 27826, 1]

In [10]:
# tokenizer.encode()
tokenizer(['That was pathetic'], padding=True, truncation=True, max_length = 120, return_tensors='pt').input_ids[0]

tensor([  466,    47,     3, 27826,     1])

In [ ]:
def translate(sentence, ):
    model.eval()

    with torch.no_grad():
        
    

In [12]:
from tqdm.notebook import tqdm

In [13]:
for i, batch in tqdm(enumerate(test_dl), total = len(test_dl)):
    print(batch[0])
    print(batch[1])

  0%|          | 0/364 [00:00<?, ?it/s]

tensor([[ 466,   47,    3,  ...,    0,    0,    0],
        [ 363,   19,    8,  ...,    0,    0,    0],
        [ 531,   25,  114,  ...,    0,    0,    0],
        ...,
        [  27,   31,   51,  ...,    0,    0,    0],
        [  27, 2065,   28,  ...,    0,    0,    0],
        [5328,   25,  114,  ...,    0,    0,    0]])
tensor([[32100,   205,    31,  ...,     0,     0,     0],
        [32100, 12281,   259,  ...,     0,     0,     0],
        [32100,    71,  5445,  ...,     0,     0,     0],
        ...,
        [32100,  1022,   140,  ...,     0,     0,     0],
        [32100,  1022,  3448,  ...,     0,     0,     0],
        [32100,  3152,    76,  ...,     0,     0,     0]])
tensor([[  100,    19,     8,  ...,     0,     0,     0],
        [   27,    31,    26,  ...,     0,     0,     0],
        [   27,    54,    31,  ...,     0,     0,     0],
        ...,
        [ 1072,    25,  9728,  ...,     0,     0,     0],
        [   27,   183,  7759,  ...,     0,     0,     0],
        [